# Data drift dashboard in jupyter notebook

## Imports

In [ ]:
import io
import numpy as np
import os
import pandas as pd
from pathlib import Path
import requests
import zipfile

from datetime import datetime
from sklearn import datasets, ensemble

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import TargetDriftPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset.regression_performance import RegressionPreset 

from evidently.metrics import (
    RegressionQualityMetric,
    RegressionPredictedVsActualScatter,
    RegressionPredictedVsActualPlot,
    RegressionErrorPlot,
    RegressionAbsPercentageErrorPlot,
    RegressionErrorDistribution,
    RegressionErrorNormality,
    RegressionTopErrorMetric,
    RegressionErrorBiasTable,
    
    DatasetSummaryMetric,
    ColumnSummaryMetric,
    DatasetMissingValuesMetric,
    DatasetCorrelationsMetric
)

## Bicycle Demand Data

### Download and extract (unzip) data

This step automatically downloads the bike dataset from UCI. This version is slightly different from the dataset used in Kaggle competition. If you want the example to be identical to the one in the Evidently blog "How to break a model in 20 days", you can manually download the dataset from Kaggle: https://www.kaggle.com/c/bike-sharing-demand/data 

And add this code:

raw_data['mnth'] = raw_data.index.map(lambda x : x.month)

raw_data['hr'] = raw_data.index.map(lambda x : x.hour)

raw_data['weekday'] = raw_data.index.map(lambda x : x.weekday() + 1)

In [ ]:
# !wget https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip -P data/

In [ ]:
# !unzip data/bike+sharing+dataset.zip -d data/

### Load data

In [ ]:
raw_data = pd.read_csv("data/hour.csv", header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [ ]:
raw_data.head()

In [ ]:
# Get weeks number
days = len(raw_data.index.unique())
weeks = days / 7

print(f'days = {days}; weeks = {weeks}')

## Regression Model

### Config

In [ ]:
REF_MONTH_START = '2011-01-01'
REF_MONTH_END = '2011-01-28'

CUR_MONTH_START = '2011-01-29'
CUR_MONTH_END = '2011-02-28'

# CUR_WEEK_START = '2011-01-29'
# CUR_WEEK_END = '2011-02-04'
# CUR_WEEK_START = '2011-02-05'
# CUR_WEEK_END = '2011-02-11'
CUR_WEEK_START = '2011-02-12'
CUR_WEEK_END = '2011-02-18'

target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

reports_dir = Path('reports') / f'{CUR_WEEK_START}_{CUR_WEEK_END}'
reports_dir.mkdir(exist_ok=True)

### Model training

In [ ]:
reference = raw_data.loc[REF_MONTH_START:REF_MONTH_END]
current = raw_data.loc[CUR_MONTH_START:CUR_MONTH_END]

In [ ]:
reference.head()

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [ ]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

# Model Monitoring

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

## Model performance

In [ ]:
regression_performance_report = Report(metrics=[RegressionPreset()])

regression_performance_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

In [ ]:
model_performance_report_path = reports_dir / 'model_performance.html'
regression_performance_report.save_html(model_performance_report_path)

##  Target drift

In [ ]:
target_drift_report = Report(metrics=[TargetDriftPreset()])
target_drift_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

In [ ]:
target_drift_report_path = reports_dir / 'target_drift.html'
target_drift_report.save_html(target_drift_report_path)

## Data drift

In [ ]:
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features

In [ ]:
data_drift_report = Report(metrics=[DataDriftPreset()])
data_drift_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

In [ ]:
data_drift_report_path = reports_dir / 'data_drift.html'
data_drift_report.save_html(data_drift_report_path)

## Data quality

In [ ]:
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features

In [ ]:
data_quality_report = Report(metrics=[DataQualityPreset()])
data_quality_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

In [ ]:
data_quality_report_path = reports_dir / 'data_quality.html'
data_quality_report.save_html(data_quality_report_path)